In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

import pickle
import os 
import time

from googleapiclient import discovery
from googleapiclient import errors


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


In [2]:
bkt_excl = 'gs://input_data_amy_bkt1/input_data/Anonymized_Fermentation_Data_final.xlsx'
#obj = open(bkt_excl, 'rb')
#obj = open('gs://input_data_amy_bkt1/input_data/Anonymized_Fermentation_Data_final_data.csv', 'rb')
data = pd.read_excel(bkt_excl,sheet_name='data') 
meta_data = pd.read_excel(bkt_excl,sheet_name='meta data') 

In [3]:
data.head()

,Product,Purpose,experiment,run,Project_Name,run_label,Strain,strain_key,Feedstock_Parent1,Start_Time,...,Cap_Oil_Em_End__percent,Cap_PCV_End__percent,Cap_Dead_Cell_Layer_End__percent,Zeex9ieJAlt_end__g_L,Zeex9ieJ_mAU_sec_end__area,Zeex9ieJ_Screening_end__g_L,Zeex9ieJ_end__g_L,Zeex9ieJ_end__g_L.1,Zeex9ieJ_end__g_L.2,interval_type
0,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,14.53607,NaN,1.673583,0.197670,2.751213,0.000000,0.150839,NaN,Cumulative
1,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,18.90319,NaN,4.333049,0.285636,3.600460,0.000000,0.294163,NaN,Cumulative
2,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,19.13803,NaN,5.186557,0.289196,3.596406,0.000000,0.325491,NaN,Cumulative
3,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,26.28490,NaN,10.083622,0.394463,4.482558,0.054673,0.277906,0.0,Cumulative
4,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,18.40111,NaN,12.402302,0.500296,5.840546,0.122632,0.301150,0.0,Cumulative


In [4]:
meta_data.head()

,index,name,description,variable type,target
0,0,Product,name of product,metadata,0
1,1,Purpose,"Fermentation purpose MF, IF, SF etc",metadata,0
2,2,experiment,Experiment ID,independent,0
3,3,run,Run ID within experiment,metadata,0
4,4,Project_Name,Project name,metadata,0


### Split Data Trainig, validation and testing